In [ ]:
from shapely.geometry import MultiPolygon, Polygon, mapping, shape
import shapely.geometry
from cv2 import filter2D
from rasterio.enums import MergeAlg
from rasterio import features
import rasterio
import whitebox
import numpy as np
import geopandas as gpd
import fiona
import os
import shutil

In [ ]:
aux_data_dir = ''
tiles = ''
faults = ''
streams = ''
dems_folder = ''
temp_folder = ''
dataset_dir = ''

geology = ''
corine = ''
soil = ''

In [ ]:
def zip_geom_attr(vector, attribute_name = None):
    out = []
    
    for record in vector:
        coords = record['geometry']['coordinates']
        for poly in coords:
            if type(poly[0]) == tuple:
                geometry = Polygon(poly)
            else: geometry = Polygon(poly[0])
            if attribute_name:
              attribute = record['properties'][attribute_name]
              pair = (geometry, attribute)
              out.append(pair)
            else:
              out.append(geometry)
    
    return out
 
def rasterise(vector, raster, output_name,  attribute_value = None, attribute = False):
    save_path = os.path.join(save_folder, output_name)
    raster = rasterio.open(os.path.join(dems_folder, raster))
    vector_path = os.path.join(temp_folder, vector)
    if os.path.exists(vector_path):
      vector = fiona.open(os.path.join(temp_folder, vector))
    
      if attribute:
        geom_value = zip_geom_attr(vector, attribute_value)
      else:
        geom_value = zip_geom_attr(vector)
      
      rasterized = rasterio.features.rasterize(geom_value,
                                    out_shape = raster.shape,
                                    transform = raster.transform,
                                    all_touched = False,
                                    fill = -999,   # background value
                                    merge_alg = MergeAlg.replace,
                                    dtype = 'int16')
    
      with rasterio.open(save_path, "w",
                       driver = "GTiff",
                       crs = raster.crs,
                       transform = raster.transform,
                       dtype = rasterio.int16,
                       count = 1,
                       width = raster.width,
                       height = raster.height) as dst:
          dst.write(rasterized, indexes = 1)

In [ ]:
def variable_buffer(vector_path, out):
  vector = gpd.read_file(vector_path)
  vector = vector[vector['Strahler'] != 1]

  distances = [5, 20, 40, 80, 200, 500, 750 ]
  conditions = [(vector['Strahler'] == 2),
                (vector['Strahler'] == 3),
                (vector['Strahler'] == 4), 
                (vector['Strahler'] == 5),
                (vector['Strahler'] == 6),
                (vector['Strahler'] == 7),
                (vector['Strahler'] == 8)]

  sthaler = vector['Strahler'].unique()

  vector['Distances'] = np.select(conditions, distances)
  vector['buffer'] = vector.buffer(vector['Distances'])
  vector = vector.drop(columns=['geometry']).set_geometry('buffer')

  if not os.path.exists(temp_folder):
    os.mkdir(temp_folder)
  if not vector.empty:
    vector.to_file(os.path.join(temp_folder, out))
  else:
    print('Stream file is empty or contains only level 1 srteams. No output generated.')

def simple_buffer(vector_path, meters, out):
  vector = gpd.read_file(os.path.join(vector_path, vector_path))
  vector['buffer'] = vector.buffer(distance = 500)
  vector = vector.drop(columns = ['geometry']).set_geometry('buffer')

  if not os.path.exists(temp_folder):
    os.mkdir(temp_folder)
  vector.to_file(os.path.join(temp_folder, out))

In [ ]:
def clip_layer(layer, overlay, out):
    layer_path = os.path.join(aux_data_dir, layer)
    overlay_path = os.path.join(tiles, overlay)
    
    overlayShp = fiona.open(overlay_path)
    overlayPoly = Polygon(overlayShp[0]['geometry']['coordinates'][0])
    
    layerShp = fiona.open(layer_path)
    layerPoly = []
    layerProperties = []
    for record in layerShp:
        poly = Polygon(record['geometry']['coordinates'][0])
        layerPoly.append(poly)
        layerProperties.append(record['properties'])
    
    clippedPoly = []
    clippedProperties = []
    for index, poly in enumerate(layerPoly):
        result = overlayPoly.intersection(poly)
        if result.area:
            clippedPoly.append(result)
            clippedProperties.append(layerProperties[index])
            
    schema = layerShp.schema
    crs = layerShp.crs

    if not os.path.exists(temp_folder):
      os.mkdir(temp_folder)
    save_path = os.path.join(temp_folder, out)
    outFile = fiona.open(save_path ,mode = 'w', driver = 'ESRI Shapefile', crs = crs, schema = schema)
    for index, poly in enumerate(clippedPoly):
        outFile.write({
            'geometry':mapping(poly),
            'properties':clippedProperties[index]
        })
    outFile.close() 

In [ ]:
def create_grid(vector, cell_size):
  bounds = vector.total_bounds
  crs = vector.crs
  xmin = bounds[0]
  ymin = bounds[1]
  xmax = bounds[2]
  ymax = bounds[3]

  grid_cells = []
  for x0 in np.arange(xmin, xmax + cell_size, cell_size ):
    for y0 in np.arange(ymin, ymax + cell_size, cell_size):
        x1 = x0-cell_size
        y1 = y0+cell_size
        grid_cells.append( shapely.geometry.box(x0, y0, x1, y1))

  cell = gpd.GeoDataFrame(grid_cells, columns=['geometry'], crs=crs)
  return cell

def density(vector_path, cell_size, out):
  vector = gpd.read_file(vector_path)
  vector['id'] = vector.index + 1

  grid = create_grid(vector, cell_size)
  grid['grid_id'] = grid.index + 1
  join = gpd.sjoin(grid, vector)

  density = join.groupby('grid_id')['id'].nunique() #Count unique storms for each grid
  density = grid.merge(density, how ='left', left_on = 'grid_id', right_index = True) #Merge the resulting geoseries with original grid dataframe
  density['id'] = density['id'].fillna(0) #Replace NA with 0 for grids with no storms
  density.rename(columns = {'id':'count'}, inplace = True)

  if not os.path.exists(temp_folder):
    os.mkdir(temp_folder)
  density.to_file(os.path.join(temp_folder, out))

In [ ]:
#Initialise whitebox and set working environment
whitebox.download_wbt(linux_musl=True, reset=True)
wbt = whitebox.WhiteboxTools()

wbt.verbose = False
print("Version information: {}".format(wbt.version()))

In [ ]:
i = 1
for dem in os.listdir(dems_folder):
    print('Processing {}/{}: {}...'.format(i, len(os.listdir(dems_folder)), dem))
    print('----------------------------------------')
    dem_path = os.path.join(dems_folder, dem)
    name = dem.split('.')[0] + '.' + dem.split('.')[1]
    save_folder = os.path.join(dataset_dir, name)

    if not os.path.exists(save_folder):
        os.mkdir(save_folder)
    wbt.work_dir = save_folder

    #Topographical modelling
    print('Breaching DEM...')
    wbt.breach_depressions(dem_path, "breached.tif")
    print('Calculating aspect...')
    wbt.aspect("breached.tif", "aspect.tif")
    print('Calculating slope...')
    wbt.slope("breached.tif", "slope.tif")
    print('Calculating ruggedness index...')
    wbt.ruggedness_index("breached.tif", "ruggedness_index.tif")
    print('Calculating profile curvature...')
    wbt.profile_curvature("breached.tif", "profile_curvature.tif")
    print('Calculating planimetric curvature...')
    wbt.plan_curvature("breached.tif", "plan_curvature.tif")
    print('Calculating tan curvature...')
    wbt.tangential_curvature("breached.tif", "tan_curvature.tif")

    #Hydrological modelling
    print('Calculating flow accumulation...')
    wbt.d8_flow_accumulation("breached.tif", "flow_accumulation.tif")
    print('Calculating sediment transport index...')
    wbt.sediment_transport_index(sca = "flow_accumulation.tif", slope = "slope.tif", output = "sediment_transport_index.tif")
    print('Calculating wetness index...')
    wbt.wetness_index(sca = "flow_accumulation.tif", slope = "slope.tif", output = "wetness_index.tif")
    print('Calculating stream power index...')
    wbt.stream_power_index(sca = "flow_accumulation.tif", slope = "slope.tif", output = "stream_power_index.tif")

    #Miscelanious
    tile = [shp for shp in os.listdir(tiles) if dem.split('.')[1] in shp and shp.endswith('.shp')][0]
    fault = [shp for shp in os.listdir(faults) if dem.split('.')[1] in shp and shp.endswith('.shp')][0]
    stream = [shp for shp in os.listdir(streams) if dem.split('.')[1] in shp and shp.endswith('.shp')][0]

    print('Clipping corine...')
    clip_layer(os.path.join(aux_data_dir, corine), tile, 'corine18.shp')
    print('Clipping geology...')
    clip_layer(os.path.join(aux_data_dir, geology), tile, 'geology.shp')
    print('Clipping soil map...')
    clip_layer(os.path.join(aux_data_dir, soil), tile, 'soil_map.shp')
    print('Creating buffer on streams...')
    variable_buffer(os.path.join(streams, stream), 'streams_distance.shp')
    print('Creating buffer on lineaments...')
    simple_buffer(os.path.join(faults, fault), 500, 'lineaments_distance.shp')
    print('Calculating lineament density...')
    density(os.path.join(faults, fault), 100, 'lineaments_density.shp')
 
    print('Rasterising vector files...')
    rasterise('corine18.shp', dem, 'corine18.tif', 'Corine_cod', attribute = True)
    rasterise('geology.shp', dem, 'geology.tif', 'Code', attribute = True)
    rasterise('soil_map.shp', dem, 'vathos.tif', 'VathosCode', attribute = True)
    rasterise('soil_map.shp', dem, 'soil_map.tif', 'Soil_code', attribute = True)
    rasterise('streams_distance.shp', dem, 'streams_distance.tif')
    rasterise('lineaments_distance.shp', dem, 'lineaments_distance.tif')
    rasterise('lineaments_density.shp', dem, 'lineaments_density.tif', 'count', attribute = True)
    print('Done!!')
    print()
    
    os.remove(os.path.join(save_folder, 'breached.tif'))
    shutil.rmtree(temp_folder)
    i = i + 1
